
# **FrankPEPstein: De Novo Peptide Generation**
### *Fragment-based generation of high-affinity peptides for protein pockets.*

**Steps:**
1.  **Setup**: Install dependencies (ADFR, OpenBabel, etc.) and download the database.
2.  **Input**: Upload your receptor and select a pocket.
3.  **Run**: Configure parameters and launch the generation pipeline.


In [ ]:

#@title 0. Install CondaColab & Setup Tools (~2 min)
import sys
import os
import subprocess
from IPython.display import clear_output

# Helper to suppress output
class SuppressStdout:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def run_setup():
    # Install tqdm first if missing (fast)
    try:
        from tqdm.notebook import tqdm
    except ImportError:
        subprocess.run("pip install -q tqdm", shell=True)
        from tqdm.notebook import tqdm

    print("Setting up FrankPEPstein environment...")
    
    steps = [
        ("Installing CondaColab", "condacolab"),
        ("Cloning Repository", "git"),
        ("Creating Conda Environment (Slow)", "env"),
        ("Configuring Notebook Utils", "patch"),
        ("Setting up External Tools (Parallel DB Download)", "tools"),
        ("Configuring Modeller", "modeller")
    ]
    
    with tqdm(total=len(steps)) as pbar:
        # 1. CondaColab
        pbar.set_description(steps[0][0])
        try:
            with SuppressStdout():
                import condacolab
                condacolab.check()
        except (ImportError, AssertionError):
            # Not installed or not ready
            print(f"\n{'='*40}")
            print("🔄 Installing CondaColab...")
            print("⚠️  The Kernel will RESTART automatically.")
            print("👉  Please RUN THIS CELL AGAIN after the restart!")
            print(f"{'='*40}\n")
            
            with SuppressStdout():
                subprocess.run("pip install -q condacolab", shell=True, check=True)
                import condacolab
                condacolab.install()
            return # Stop execution here as kernel dies
        pbar.update(1)

        # 2. Git Clone
        pbar.set_description(steps[1][0])
        with SuppressStdout():
            if not os.path.exists("FrankPEPstein"):
                subprocess.run("git clone https://github.com/Joacaldog/FrankPEPstein.git", shell=True, check=True)
        pbar.update(1)

        # 3. Create Environment
        pbar.set_description(steps[2][0])
        env_path = "/usr/local/envs/FrankPEPstein"
        if not os.path.exists(env_path):
             # Added 'pigz' to the package list
             subprocess.run("mamba create -n FrankPEPstein -q -y -c conda-forge -c salilab openbabel biopython fpocket joblib tqdm py3dmol vina pigz python=3.10 salilab::modeller > /dev/null 2>&1", shell=True, check=True)
        
        # Configure Path
        site_packages = f"{env_path}/lib/python3.10/site-packages"
        if site_packages not in sys.path:
            sys.path.append(site_packages)
        os.environ['PATH'] = f"{env_path}/bin:" + os.environ['PATH']
        pbar.update(1)

        # 4. Patch Utils
        pbar.set_description(steps[3][0])
        # We write a clean utils file
        patched_utils_content = r'''import os
import sys
import glob
import shutil
import subprocess

def configure_modeller(license_key='MODELIRANJE', repo_dir='FrankPEPstein'):
    template_config = os.path.join(repo_dir, "utilities/config.py")
    dest_config = None
    try:
        import modeller
        modeller_path = os.path.dirname(modeller.__file__)
        candidate = os.path.join(modeller_path, "config.py")
        if os.path.exists(candidate):
            dest_config = candidate
    except Exception:
        pass

    if not dest_config:
        possible_paths = [
            f"{sys.prefix}/lib/modeller-*/modlib/modeller/config.py",
            f"{sys.prefix}/lib/python*/site-packages/modeller/config.py",
            "/usr/local/envs/FrankPEPstein/lib/modeller-*/modlib/modeller/config.py"
        ]
        for pattern in possible_paths:
            found = glob.glob(pattern)
            if found:
                dest_config = found[0]
                break

    if dest_config and os.path.exists(template_config):
        with open(template_config, 'r') as f:
            content = f.read()
        new_content = content.replace("'MODELIRANJE'", f"'{license_key}'")
        with open(dest_config, 'w') as f:
            f.write(new_content)
        return True
    return False

def setup_external_tools(drive_ids=None):
    if drive_ids is None: drive_ids = {}
    
    # Install gdown if needed
    try: import gdown
    except ImportError: subprocess.run([sys.executable, "-m", "pip", "install", "gdown"], check=True); import gdown

    # Determine Base Dir
    base_dir = "FrankPEPstein" if os.path.exists("FrankPEPstein") else "."
    utilities_dir = os.path.join(base_dir, "utilities")
    db_dir = os.path.join(base_dir, "DB")
    os.makedirs(utilities_dir, exist_ok=True)
    os.makedirs(db_dir, exist_ok=True)


    # 1. Database (Parallel Download & Decompress)
    db_id = drive_ids.get("db_id")
    # Determine pigz availability
    use_pigz = True if subprocess.run("which pigz", shell=True).returncode == 0 else False
    extract_flag = "-I pigz -xf" if use_pigz else "-xzf"

    db_tar = os.path.join(base_dir, "filtered_DB_P5-15_R30_id10_optim.tar.gz")
    
    # Check if Main DB is already populated (simple check)
    if not os.path.exists(os.path.join(db_dir, "filtered_DB_P5-15_R30_id10")):
        if db_id:
             if not os.path.exists(db_tar):
                 print(f"Downloading DB (ID: {db_id})...")
                 # We assume single big tarball for now based on user context
                 gdown.download(f'https://drive.google.com/uc?id={db_id}', db_tar, quiet=True)
             
             if os.path.exists(db_tar):
                 print(f"Extracting DB using {extract_flag}...")
                 subprocess.run(f"tar {extract_flag} {db_tar} -C {db_dir} > /dev/null 2>&1", shell=True, check=True)
                 # Optional: delete tar to save space?
                 # os.remove(db_tar)

    # 1.5 Minipockets Database
    mini_id = drive_ids.get("minipockets_id")
    mini_tar = os.path.join(base_dir, "minipockets.tar.gz")
    # We check for a known folder inside, e.g. "minipockets_surface80_winsize3_size3_curated"
    # Or just check if the tar was extracted.
    # User said "-fm" points to it.
    # Let's assume the tar extracts to a folder inside DB.
    
    if mini_id:
        # Check standard folder name (guess or generic)
        # If user didn't specify name, we rely on tar content.
        # But we need a check to avoid redownload.
        # Let's check if 'minipockets*' exists in DB dir
        existing_mini = glob.glob(os.path.join(db_dir, "minipockets*"))
        if not existing_mini:
             if not os.path.exists(mini_tar):
                  print(f"Downloading Minipockets DB (ID: {mini_id})...")
                  gdown.download(f'https://drive.google.com/uc?id={mini_id}', mini_tar, quiet=True)
             
             if os.path.exists(mini_tar):
                  print(f"Extracting Minipockets DB...")
                  subprocess.run(f"tar {extract_flag} {mini_tar} -C {db_dir} > /dev/null 2>&1", shell=True, check=True)

    # 2. Utilities (ADFR)
    # Usually passed as zip or tar
    # User might have "utilities_pkg_id"
    util_id = drive_ids.get("utilities_pkg_id")
    util_tar = os.path.join(base_dir, "utilities.tar.gz")
    
    if util_id and not os.path.exists(os.path.join(utilities_dir, "ADFRsuite_x86_64Linux_1.0")):
         if not os.path.exists(util_tar):
              gdown.download(f'https://drive.google.com/uc?id={util_id}', util_tar, quiet=True)
         subprocess.run(f"tar {extract_flag} {util_tar} -C {utilities_dir} > /dev/null 2>&1", shell=True, check=True)

    # ADFR Installation / License Check
    # Look for install script
    adfr_root = os.path.join(utilities_dir, "ADFRsuite_x86_64Linux_1.0")
    install_sh = os.path.join(utilities_dir, "install.sh") # Common location? Or inside extracted folder?
    
    # Sometimes it extracts AS "ADFRsuite..." directly.
    # If the user provides a zip of the INSTALLED directory, no installation needed.
    # Checks bin
    adfr_bin = os.path.join(adfr_root, "bin")
    if os.path.exists(adfr_bin):
        # Add to PATH
        if adfr_bin not in os.environ['PATH']:
            os.environ['PATH'] += f":{adfr_bin}"
            print(f"Added ADFR bin to PATH: {adfr_bin}")
    else:
        # If bin missing, maybe we need to run install?
        # Check for an install script in utilities
        candidates = glob.glob(os.path.join(utilities_dir, "*install*.sh"))
        if candidates:
             installer = candidates[0]
             print(f"Running ADFR installer: {installer}")
             # Pipe 'yes' to accept license
             # usage: yes | ./install.sh -d target_dir
             # or simply ./install.sh and expect yes
             # We assume it asks "Do you accept... (y/n)"
             subprocess.run(f"yes | bash {installer}", shell=True) # Try bash execution
             
             # Recheck bin
             if os.path.exists(adfr_bin):
                  os.environ['PATH'] += f":{adfr_bin}"

    # Click
    click_bin = os.path.join(utilities_dir, "Click/bin") 
    if os.path.exists(click_bin):
        if click_bin not in os.environ['PATH']:
             os.environ['PATH'] += f":{click_bin}"
        subprocess.run(f"chmod +x {click_bin}/click", shell=True)

'''
        os.makedirs("FrankPEPstein/scripts", exist_ok=True)
        with open("FrankPEPstein/scripts/notebook_utils.py", "w") as f:
            f.write(patched_utils_content)
        pbar.update(1)

        # 5. External Tools Setup
        pbar.set_description(steps[4][0])
        repo_path = os.path.abspath("FrankPEPstein")
        if repo_path not in sys.path:
            sys.path.append(repo_path)
        from scripts import notebook_utils
        
        # Correct Drive IDs
        drive_ids = {
            "db_id": "13a6M_UVham9SiBCE6PCQQi6CnUvGNvNO", # Provided by user
            "minipockets_id": "1a4GoZ1ZT-DNYMyvVtKJukNdF6TAaLJU5", # New Minipockets DB
            "utilities_pkg_id": "1gmRj8mva84-JB7UXUcQfB3Ziw_nwwdox", # Kept existing
            # If explicit ADFR installer needed:
            # "adfr_installer_id": "..."
        }
        
        with SuppressStdout():
             notebook_utils.setup_external_tools(drive_ids)
        pbar.update(1)

        # 6. Configure Modeller
        pbar.set_description(steps[5][0])
        with SuppressStdout():
            notebook_utils.configure_modeller()
        pbar.update(1)
        
    clear_output()
    print("✅ Setup Ready!")

if __name__ == "__main__":
    run_setup()



In [ ]:
#@title 1. Input & Pocket Selection
#@markdown **Instructions:**
#@markdown 1. Upload your Receptor PDB.
#@markdown 2. Choose Mode: **Auto Detect** (runs fpocket) or **Manual Upload** (upload your specific pocket PDB).
#@markdown 3. Select the pocket from the dropdown to visualize.

import os
import sys
import subprocess
try:
    import py3Dmol
except ImportError:
    # Try adding FrankPEPstein env to path
    env_path = "/usr/local/envs/FrankPEPstein"
    site_packages = f"{env_path}/lib/python3.10/site-packages"
    if os.path.exists(site_packages):
        if site_packages not in sys.path:
            sys.path.append(site_packages)
        # Add binary path too
        if f"{env_path}/bin" not in os.environ['PATH']:
            os.environ['PATH'] = f"{env_path}/bin:" + os.environ['PATH']
    
    # Retry import
    try:
        import py3Dmol
    except ImportError:
        print("py3Dmol not found. Installing...")
        subprocess.run("pip install -q py3dmol", shell=True, check=True)
        import py3Dmol

import ipywidgets as widgets
from google.colab import files
from IPython.display import display

# --- configuration ---
detection_mode = "Auto Detect" #@param ["Auto Detect", "Manual Upload"]

# Global variables for next steps
receptor_filename = None
pockets_dir = "pockets_upload" # Default for manual
final_pockets_list = []

# --- Persistence Logic ---
import json

def save_pipeline_state(updates):
    state_file = "pipeline_state.json"
    current_state = {}
    if os.path.exists(state_file):
        try:
            with open(state_file, "r") as f:
                current_state = json.load(f)
        except:
            pass
    current_state.update(updates)
    with open(state_file, "w") as f:
        json.dump(current_state, f, indent=4)
    print(f"State saved to {state_file}")

# --- 1. Upload Receptor ---
print(f"--- Upload Receptor PDB ({detection_mode}) ---")
uploaded_r = files.upload()

import re

if not uploaded_r:
    print("No receptor file uploaded.")
else:
    raw_filename = list(uploaded_r.keys())[0]
        
    # Check for Colab duplicate naming (e.g. receptor(1).pdb)
    match = re.search(r'^(.*?)\s?\(\d+\)(\.[^.]*)?$', raw_filename)
    if match:
        clean_name = match.group(1) + (match.group(2) if match.group(2) else "")
        print(f"Detected duplicate upload: {raw_filename} -> overwriting {clean_name}")
        
        if os.path.exists(clean_name):
            os.remove(clean_name)
        os.rename(raw_filename, clean_name)
        receptor_filename = os.path.abspath(clean_name)
    else:
        receptor_filename = os.path.abspath(raw_filename)
        
    print(f"Receptor: {receptor_filename}")
    save_pipeline_state({"receptor_filename": receptor_filename})

    # --- 2. Pocket Handling ---
    if detection_mode == "Auto Detect":
        print(f"\nRunning fpocket on {receptor_filename}...")
        try:
            # Fix: Quotes for filenames with spaces
            subprocess.run(f"fpocket -f '{receptor_filename}'", shell=True, check=True)
            
            # Robust folder finding
            # fpocket creates output based on the filename in the SAME directory
            # receptor_filename is absolute, so output should be absolute too?
            # fpocket output format: /path/to/file_out/
            
            base_name_no_ext = os.path.splitext(os.path.basename(receptor_filename))[0]
            base_dir = os.path.dirname(receptor_filename)
            
            # Possible output folder names
            folder_name_1 = f"{os.path.basename(receptor_filename)}_out"
            folder_name_2 = f"{base_name_no_ext}_out"
            
            # Check in the same directory as the receptor
            possible_folders = [
                os.path.join(base_dir, folder_name_1),
                os.path.join(base_dir, folder_name_2)
            ]
            
            output_folder = next((f for f in possible_folders if os.path.exists(f)), None)

            if output_folder:
                pockets_dir = os.path.join(output_folder, "pockets")
                if os.path.exists(pockets_dir):
                    final_pockets_list = [f for f in os.listdir(pockets_dir) if f.endswith(".pdb")]
                    print(f"Auto-detection finished. Found {len(final_pockets_list)} pockets.")
                    pockets_dir = os.path.abspath(pockets_dir) # Ensure absolute
                else:
                    print(f"Warning: pockets subdirectory not found in {output_folder}")
            else:
                print("Error: fpocket output not found. Checked:", possible_folders)
                
        except subprocess.CalledProcessError:
             print("Error running fpocket.")

    elif detection_mode == "Manual Upload":
        print(f"\n--- Upload Manual Pocket PDB ---")
        os.makedirs(pockets_dir, exist_ok=True)
        uploaded_p = files.upload()
        import re
        if uploaded_p:
            for p_file in uploaded_p.keys():
                # Colab renames duplicate uploads to filename(1).ext. 
                # User wants to overwrite instead.
                
                # Check for pattern like "name(1).pdb" or "name (1).pdb"
                # Regex matches: (any content) optional space (digits) (extension)
                match = re.search(r'^(.*?)\s?\(\d+\)(\.[^.]*)?$', p_file)
                
                if match:
                    clean_name = match.group(1) + (match.group(2) if match.group(2) else "")
                    print(f"Detected duplicate upload: {p_file} -> overwriting {clean_name}")
                else:
                    clean_name = p_file

                target_path = os.path.join(pockets_dir, clean_name)
                
                # If target exists, log that we are replacing it
                if os.path.exists(target_path):
                    print(f"Replacing existing file: {clean_name}")
                    os.remove(target_path)
                
                # Move (rename) the uploaded file to the target path
                # Note: 'p_file' is in CWD (content/), target is in pockets_dir
                os.rename(p_file, target_path)
                
                if clean_name not in final_pockets_list:
                    final_pockets_list.append(clean_name)
                    
            print(f"Manual upload finished. Available pockets: {len(final_pockets_list)}")

    # --- 3. Visualization & Selection ---
    if final_pockets_list:
        print("\n--- Pocket Selection & Visualization ---")
        print("Displaying all detected pockets. Select one below for extraction.")
        
        pocket_dropdown = widgets.Dropdown(
            options=sorted(final_pockets_list),
            description='Select Pocket:',
            disabled=False,
        )

        def view_pockets(selected_pocket_file):
            view = py3Dmol.view(width=800, height=600)
            
            # 1. Receptor Surface (White, Transparent)
            with open(receptor_filename, 'r') as f:
                view.addModel(f.read(), "pdb")
            view.setStyle({}) 
            view.addSurface(py3Dmol.SES, {'opacity': 0.3, 'color': 'white'})
            
            # 2. Add ALL pockets with distinct colors
            colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#FFA500', '#800080', '#008000', '#800000']
            
            # Regex to find number in filename (e.g. pocket5_atm.pdb -> 5, or just 5.pdb -> 5)
            import re
            
            for i, p_file in enumerate(sorted(final_pockets_list)):
                full_path = os.path.join(pockets_dir, p_file)
                if os.path.exists(full_path):
                    with open(full_path, 'r') as f:
                        view.addModel(f.read(), "pdb")
                    
                    # Try to extract a short label (number)
                    # Common patterns: "pocket5_atm.pdb", "pocket5.pdb", "5.pdb"
                    match = re.search(r'(\d+)', p_file)
                    label_text = match.group(1) if match else p_file
                    
                    # Determine styling
                    is_selected = (p_file == selected_pocket_file)
                    
                    if is_selected:
                         color = 'red'
                         opacity = 1.0
                         label_style = {'fontSize': 18, 'fontColor': 'red', 'backgroundColor': 'white', 'backgroundOpacity': 0.8, 'border': '2px solid red'}
                    else:
                        color = colors[i % len(colors)]
                        opacity = 0.6
                        label_style = {'fontSize': 12, 'fontColor': 'black', 'backgroundColor': 'white', 'backgroundOpacity': 0.5}

                    view.setStyle({'model': -1}, {'sphere': {'color': color, 'opacity': opacity}})
                    
                    # Add 3D Label (Number only)
                    view.addLabel(label_text, label_style, {'model': -1})

            view.zoomTo()
            view.show()
            
        display(widgets.interactive(view_pockets, selected_pocket_file=pocket_dropdown))
    else:
        print("No pockets available to select.")
#@title 4. Pocket Extraction & Box Generation
#@markdown This step extracts the selected pocket (residues within 5Å of fpocket spheres) and calculates the grid box.

import os
import subprocess # Added subprocess import here for consistency
import sys # Added sys import for python_exe fallback

# --- Helper Functions (Subprocess) ---
def run_extraction_and_box_isolated(receptor_path, pocket_path, output_pocket_path, buffer=5.0):
    """
    Runs extraction (NeighborSearch) and box calculation in isolated environment.
    """
    
    script_content = """
import sys
import os
from Bio import PDB
from Bio.PDB import PDBParser, PDBIO, Select, NeighborSearch

def extract_and_box(receptor_file, pocket_atm_file, output_file, buffer_val):
    try:
        parser = PDBParser(QUIET=True)
        
        # 1. Load Structures
        # print(f"Loading receptor: {receptor_file}")
        receptor_struct = parser.get_structure("receptor", receptor_file)
        
        # print(f"Loading pocket atoms: {pocket_atm_file}")
        pocket_struct = parser.get_structure("pocket_atm", pocket_atm_file)
        
        # 2. Collect Pocket Atoms (e.g. Alpha Spheres)
        pocket_atoms = []
        for model in pocket_struct:
            for chain in model:
                for residue in chain:
                    for atom in residue:
                        pocket_atoms.append(atom)
        
        if not pocket_atoms:
            print("ERROR: No atoms in pocket file")
            return

        # 3. Neighbor Search (5.0 Angstrom)
        # Find PROTEIN residues near the pocket spheres
        receptor_atoms = list(receptor_struct.get_atoms())
        ns = NeighborSearch(receptor_atoms)
        
        selected_residues = set()
        
        for p_atom in pocket_atoms:
            # Level 'R' returns residues
            nearby_residues = ns.search(p_atom.get_coord(), 5.0, level='R')
            for res in nearby_residues:
                selected_residues.add((res.parent.id, res.id))
                
        # 4. Save Extracted Pocket (Chain 'p')
        class PocketSelect(Select):
            def accept_residue(self, residue):
                return (residue.parent.id, residue.id) in selected_residues

        io = PDBIO()
        io.set_structure(receptor_struct)
        
        # We need to save, but also RENAME chain to 'p'
        # To do this cleanly without modifying original struct in memory too much:
        # Save to temp, reload, rename, save final.
        
        temp_out = output_file + ".tmp"
        io.save(temp_out, PocketSelect())
        
        # Reload temp to verify and calculate box
        extracted_struct = parser.get_structure("extracted", temp_out)
        
        all_coords = []
        for model in extracted_struct:
            for chain in model:
                chain.id = 'p' # RENAME TO p
                for residue in chain:
                    for atom in residue:
                        all_coords.append(atom.get_coord())
        
        # Save Final
        io.set_structure(extracted_struct)
        io.save(output_file)
        os.remove(temp_out)
        
        # 5. Box Calculation
        if not all_coords:
            print("ERROR: No residues selected")
            return

        min_coord = [min([c[i] for c in all_coords]) for i in range(3)]
        max_coord = [max([c[i] for c in all_coords]) for i in range(3)]
        
        # Basic Center
        center = [(min_coord[i] + max_coord[i]) / 2 for i in range(3)]
        
        # Size + Buffer
        # buffer_val is passed (usually 10 to add padding)
        # User requested trick? Standard padding is usually sufficient if 5A extraction was done.
        size = [(max_coord[i] - min_coord[i]) + float(buffer_val) for i in range(3)]
        
        print(f"CENTER:{center[0]},{center[1]},{center[2]}")
        print(f"SIZE:{size[0]},{size[1]},{size[2]}")
        print("SUCCESS")

    except Exception as e:
        print(f"ERROR:{e}")

if __name__ == "__main__":
    extract_and_box(sys.argv[1], sys.argv[2], sys.argv[3], float(sys.argv[4]))

"""
    script_name = "extract_box_isolated.py"
    with open(script_name, "w") as f:
        f.write(script_content)
        
    python_exe = "/usr/local/envs/FrankPEPstein/bin/python"
    # Fallback to sys.executable if env not found (local testing)
    if not os.path.exists(python_exe): python_exe = sys.executable

    try:
        result = subprocess.run(
            [python_exe, script_name, receptor_path, pocket_path, output_pocket_path, str(buffer)],
            capture_output=True, text=True, check=True
        )
        
        center, size = None, None
        success = False
        
        for line in result.stdout.splitlines():
            if line.startswith("CENTER:"):
                center = [float(x) for x in line.split(":")[1].split(",")]
            elif line.startswith("SIZE:"):
                size = [float(x) for x in line.split(":")[1].split(",")]
            elif line.startswith("SUCCESS"):
                success = True
            elif line.startswith("ERROR"):
                print(f"Script Error: {line}")
                
        return center, size, success

    except subprocess.CalledProcessError as e:
        print(f"Execution Error: {e.stderr}")
        return None, None, False

# --- GUI ---
output_log = widgets.Output()

def extract_and_calculate_box(b):
    output_log.clear_output()
    with output_log:
        if 'pocket_dropdown' not in globals() or not pocket_dropdown.value:
            print("No pocket selected.")
            return
        
        if 'receptor_filename' not in globals() or not receptor_filename:
             print("Receptor not defined.")
             return

        selected_pocket = pocket_dropdown.value
        pocket_path = os.path.join(pockets_dir, selected_pocket)
        
        # Define output path for the extracted (real) pocket
        # stored in the same dir as receptor usually, or pockets dir
        extracted_filename = "pocket.pdb"
        extracted_path = os.path.join(pockets_dir, extracted_filename)
        
        print(f"Processing {selected_pocket}...")
        print(f"Receptor: {receptor_filename}")
        print("Running NeighborSearch (5Å) + Box Calculation...")
        
        center, size, success = run_extraction_and_box_isolated(
            receptor_filename, pocket_path, extracted_path, buffer=10.0 # Buffer for BOX size (padding), extraction radius is 5.0 inside script
        )
        
        if success and center:
            center_str = f"{center[0]:.3f}, {center[1]:.3f}, {center[2]:.3f}"
            size_str = f"{size[0]:.3f}, {size[1]:.3f}, {size[2]:.3f}"
            
            print("-" * 30)
            print(f"Box Center: {center_str}")
            print(f"Box Size:   {size_str}")
            print("-" * 30)
            print(f"✅ Created Extracted Pocket: {extracted_path} (pocket.pdb)")
            
            global box_center, box_size, extracted_pocket_path
            box_center = center
            box_size = size
            extracted_pocket_path = os.path.abspath(extracted_path)
            
            save_pipeline_state({
                "box_center": box_center,
                "box_size": box_size,
                "extracted_pocket_path": extracted_pocket_path
            })
        else:
            print("Extraction Failed.")

extract_btn = widgets.Button(
    description='Extract Pocket & Calculate Box',
    button_style='success',
    icon='box',
    layout=widgets.Layout(width='50%')
)
extract_btn.on_click(extract_and_calculate_box)

print("\n--- 4. Extraction Control ---")
display(extract_btn, output_log)



In [ ]:
#@title 2. FrankPEPstein Pipeline
#@markdown **Instructions:**
#@markdown 1. Configure parameters.
#@markdown 2. Run to start the pipeline. Real-time visualization will appear below.

import os
import sys
import subprocess
import shutil
import time
import threading
import ipywidgets as widgets
from IPython.display import display, Javascript

# --- Configuration Widgets ---
style = {'description_width': 'initial'}

length_slider = widgets.IntSlider(
    value=10, min=3, max=30, step=1,
    description='Target Peptide Length:', style=style, layout=widgets.Layout(width='50%')
)
num_peptides_slider = widgets.IntSlider(
    value=10, min=1, max=100, step=1,
    description='Number of Output Peptides:', style=style, layout=widgets.Layout(width='50%')
)
threads_slider = widgets.IntSlider(
    value=os.cpu_count() or 2, min=1, max=os.cpu_count() or 8, step=1,
    description='CPU Threads:', style=style, layout=widgets.Layout(width='50%')
)

run_output = widgets.Output()

# --- Visualization Helper ---
def update_viz_js(mol_view, fragment_pdbs):
    """
    Generates JS to update the viewer with new fragments without full reload.
    """
    # This is tricky in Colab. Usually we re-render the 3D mol.
    # For simplicity, we might just re-render the view every X seconds if new files found.
    pass

# --- Pipeline Logic ---
import json

def run_frankpepstein_pipeline(b):
    run_output.clear_output()
    with run_output:
        # Load State
        state_file = "pipeline_state.json"
        global box_center, box_size, extracted_pocket_path, receptor_filename
        
        if os.path.exists(state_file):
            try:
                with open(state_file, "r") as f:
                    state = json.load(f)
                if 'box_center' in state: box_center = state['box_center']
                if 'box_size' in state: box_size = state['box_size']
                if 'extracted_pocket_path' in state: extracted_pocket_path = state['extracted_pocket_path']
                if 'receptor_filename' in state: receptor_filename = state['receptor_filename']
            except: pass

        if 'box_center' not in globals() or 'extracted_pocket_path' not in globals():
            print("Error: Missing pipeline state (Pocket/Box). Run Step 1/4 first.")
            return

        # Setup Paths
        base_dir = os.path.abspath("FrankPEPstein") if os.path.exists("FrankPEPstein") else os.path.abspath(".")
        scripts_dir = os.path.join(base_dir, "scripts")
        # DB Path: filtered_DB_P5-15_R30_id10 (extracted from ...optim.tar.gz)
        db_path = os.path.join(base_dir, "DB", "filtered_DB_P5-15_R30_id10") 
        complexes_path = db_path # User says this is the path for -d too? Or inside?
        # User said: "la carpeta que se pone como -d es filtered_DB_P5-15_R30_id10"
        # Superposer requires "-d" (database path) and "-fm" (folder minipockets? or also DB?)
        # Step 2 script sets -d to complexes_path and -fm to db_path. 
        # I will set both to this directory.
        complexes_path = db_path
        
        # Fallback for complexes if not extracted yet (User mentioned parallel download)
        # But for now we assume it exists
        
        run_dir = os.path.join(base_dir, "FrankPEPstein_Run")
        if os.path.exists(run_dir): shutil.rmtree(run_dir)
        os.makedirs(run_dir)
        
        print(f"Initializing Run in {run_dir}...")
        
        # 0. Copy Receptor
        target_receptor = "receptor.pdb"
        try:
            shutil.copy(receptor_filename, os.path.join(run_dir, target_receptor))
        except:
            print("Error copying receptor.")
            return

        # 1. Prepare Pocket (Rename Chain 'p')
        print("Preparing Pocket...")
        prep_script = os.path.join(run_dir, "prep_pocket.py")
        target_pocket = "target_pocket.pdb"
        
        with open(prep_script, "w") as f:
            f.write(f"""
import sys
from Bio import PDB
try:
    parser = PDB.PDBParser(QUIET=True)
    s = parser.get_structure("p", "{extracted_pocket_path}")
    for m in s:
        for c in m: c.id = 'p'
    io = PDB.PDBIO()
    io.set_structure(s)
    io.save("{os.path.join(run_dir, target_pocket)}")
    print("SUCCESS")
except Exception as e: print(e)
""")
        
        # Run with env python
        frank_python = "/usr/local/envs/FrankPEPstein/bin/python"
        subprocess.run([frank_python, prep_script], check=True)
        
        if not os.path.exists(os.path.join(run_dir, target_pocket)):
            print("Pocket preparation failed.")
            return

        # 2. Superposer (Step 1)
        print("\nStep 1: Fragment Scanning (Superposer)...")
        # Ensure 'click' and 'adfr' in PATH
        env = os.environ.copy()
        
        # Find paths dynamically
        utilities_dir = os.path.join(base_dir, "utilities")
        
        # Determine Minipockets DB (starts with minipockets...)
        db_root = os.path.dirname(db_path) # parent of filtered_DB
        minipockets_path = db_path # Fallback
        try:
            candidates = [d for d in os.listdir(db_root) if d.startswith("minipockets")]
            if candidates:
                minipockets_path = os.path.join(db_root, candidates[0])
        except:
             pass

        # Add Click
        click_bin = os.path.join(utilities_dir, "Click/bin")
        if os.path.exists(click_bin): env["PATH"] = f"{click_bin}:{env['PATH']}"
        
        # Add ADFR
        adfr_bin = os.path.join(utilities_dir, "ADFRsuite_x86_64Linux_1.0/bin") # Common struct
        if os.path.exists(adfr_bin): env["PATH"] = f"{adfr_bin}:{env['PATH']}"

        superposer_script = os.path.join(scripts_dir, "superposerV5.2_leave1out_cluster.py")
        
        cmd_super = [
            frank_python, superposer_script,
            "-T", target_pocket,
            "-d", complexes_path,
            "-a", "3", "-r", "0.1",
            "-x_center", str(box_center[0]), "-y_center", str(box_center[1]), "-z_center", str(box_center[2]),
            "-x_size", str(box_size[0]), "-y_size", str(box_size[1]), "-z_size", str(box_size[2]),
            "-t", str(threads_slider.value),
            "-fm", minipockets_path
        ]
        
        # Start Visualization Thread
        # We will monitor the output folder of superposer
        # output folder format: superpockets_residuesAligned3_RMSD0.1
        super_out_dir = os.path.join(run_dir, "superpockets_residuesAligned3_RMSD0.1")
        
        # viz_monitor writes to global viz_output checking run folder
        
        stop_event = threading.Event()
        
        def viz_monitor():
            import py3Dmol
            import time
            from IPython.display import display, clear_output
            
            # Initial Wait
            time.sleep(2)
            
            seen_fragments = 0
            
            while not stop_event.is_set():
                # 1. Check for fragments
                fragments = []
                if os.path.exists(super_out_dir):
                    fragments = [f for f in os.listdir(super_out_dir) if f.startswith("patch_file")]
                
                n_frags = len(fragments)
                
                # Update only if new fragments found or first run
                if n_frags > seen_fragments or seen_fragments == 0:
                     seen_fragments = n_frags
                     
                     with viz_output:
                        clear_output(wait=True)
                        print(f"Visualization Update: {n_frags} fragments found.")
                        
                        view = py3Dmol.view(width=800, height=600)
                        
                        # A. Receptor (Defined, not transparent)
                        view.addModel(open(receptor_filename, 'r').read(), "pdb")
                        view.setStyle({'model': 0}, {'cartoon': {'color': 'white'}}) 
                        # Optional: Add faint surface ? User asked for defined.
                        # view.addSurface(py3Dmol.SES, {'opacity': 0.5, 'color': 'white'}, {'model': 0})
                        
                        # B. Box (Red, Defined)
                        view.addBox({
                            'center': {'x': box_center[0], 'y': box_center[1], 'z': box_center[2]},
                            'dimensions': {'w': box_size[0], 'h': box_size[1], 'd': box_size[2]},
                            'color': 'red',
                            'opacity': 0.6,
                            'wireframe': True
                        })
                        
                        # C. Fragments
                        import random
                        viz_frags = fragments
                        if len(viz_frags) > 50:
                             viz_frags = random.sample(fragments, 50)
                             
                        for frag_file in viz_frags:
                            frag_path = os.path.join(super_out_dir, frag_file)
                            try:
                                view.addModel(open(frag_path, 'r').read(), "pdb")
                                view.setStyle({'model': -1}, {'stick': {'colorscheme': 'greenCarbon'}})
                            except: pass
                            
                        view.zoomTo()
                        view.show()
                
                time.sleep(5)
                
        viz_thread = threading.Thread(target=viz_monitor)
        viz_thread.start()
        
        try:
            # Run Superposer
            if not os.path.exists(complexes_path):
                 print(f"Warning: Complex DB not found at {complexes_path}")
            
            # Print Command
            print("\nRunning Superposer with command:")
            print(" ".join(cmd_super))
            print("-" * 20 + "\n")

            process = subprocess.Popen(cmd_super, cwd=run_dir, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)
            
            for line in iter(process.stdout.readline, ''):
                print(line, end='')
                
            process.wait()
            
            stop_event.set()
            viz_thread.join()
            
            print("\nSuperposer Completed.")
            
        except Exception as e:
            stop_event.set()
            print(f"Error: {e}")
            return

        # 3. Patch Clustering (Step 2 & 3)
        # Note: The original pipeline split this into FrankVINA I (filtering) -> PatchClust -> FrankVINA II (scoring)
        # We will follow that.
        
        pep_len = length_slider.value
        n_peps = num_peptides_slider.value
        
        # A. Filter (FrankVINA I)
        print("\nStep 2: Patch Filtering...")
        if not os.path.exists(super_out_dir):
            print("No Superposer output.")
            return

        # Copy receptor to output dir for Vina
        shutil.copy(os.path.join(run_dir, target_receptor), os.path.join(super_out_dir, target_receptor))
        
        vina_script_1 = os.path.join(scripts_dir, "frankVINA_FNKPSTN_cluster.py")
        subprocess.run([frank_python, vina_script_1, target_receptor, str(threads_slider.value)], cwd=super_out_dir, check=True, env=env)
        
        # B. Cluster
        print("\nStep 3: Clustering...")
        patches_dir = os.path.join(super_out_dir, "top_10_patches")
        if not os.path.exists(patches_dir):
            print("No qualified patches found.")
            return
            
        clust_script = os.path.join(scripts_dir, "patch_clustering_V8.7_cluster.py")
        subprocess.run([frank_python, clust_script, "-w", str(pep_len), "-t", str(threads_slider.value)], cwd=patches_dir, check=True, env=env)
        
        # C. Score (FrankVINA II)
        print("\nStep 4: Final Scoring...")
        final_dir = os.path.join(patches_dir, f"frankPEPstein_{pep_len}")
        if not os.path.exists(final_dir):
             print("Clustering failed.")
             return
        
        shutil.copy(os.path.join(run_dir, target_receptor), os.path.join(final_dir, target_receptor))
        vina_script_2 = os.path.join(scripts_dir, "frankVINA_V3_cluster.py")
        subprocess.run([frank_python, vina_script_2, target_receptor, str(threads_slider.value), str(n_peps)], cwd=final_dir, check=True, env=env)
        
        print("\nPipeline Finished!")
        # Show Top Results?
        results_file = os.path.join(final_dir, f"top_{n_peps}_peps", f"top{n_peps}_peps.tsv")
        if os.path.exists(results_file):
             print(f"Results saved to: {results_file}")
             # Display top 5
             with open(results_file) as f:
                 print("\nTop Candidates:")
                 print(f.read())

run_btn = widgets.Button(description='Run Pipeline', button_style='danger', icon='rocket')
run_btn.on_click(run_frankpepstein_pipeline)


viz_output = widgets.Output()

display(widgets.VBox([
    widgets.HBox([length_slider, num_peptides_slider]),
    threads_slider,
    run_btn,
    run_output,
    viz_output
]))

